<h2>Setup AWS Cognito</h2>

In [1]:
import sys
import os

print(sys.executable)
# Get the current notebook's directory
#current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
current_dir = os.getcwd()

utils_dir = os.path.join(current_dir, 'utils')
utils_dir = os.path.abspath(utils_dir)
print(utils_dir)
# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\.venv\Scripts\python.exe
c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\utils
sys.path[0]: c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\utils


In [2]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Setting up Amazon Cognito user pool...
Region from profile: us-east-1
Pool id: us-east-1_6Xo1XWJao
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_6Xo1XWJao/.well-known/openid-configuration
Client ID: 5rd1b1lqb0bujcs5k5p4dmgd97
Bearer Token: eyJraWQiOiJ2aXN5TWIwZjZsTlRWUjRoWFFvekVhcnI2dGdyZlIwQ1JqbVM2NGtJN05RPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIyNGE4ZTQ5OC01MDYxLTcwMTQtNjcxMi1mZWYzMjc3OTdiNzgiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV82WG8xWFdKYW8iLCJjbGllbnRfaWQiOiI1cmQxYjFscWIwYnVqY3M1azVwNGRtZ2Q5NyIsIm9yaWdpbl9qdGkiOiJjYmZlNTU1Zi0wMTAwLTQwNWMtYjRhMC1kZmJkNTdlYWM2ZGMiLCJldmVudF9pZCI6IjAzOGVmOWEyLTdhNjUtNDFlMy1hMTc3LTNiNzU3MDhlNzYxZiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NTY4ODcwODMsImV4cCI6MTc1Njg5MDY4MywiaWF0IjoxNzU2ODg3MDgzLCJqdGkiOiJhZmMwNjZlZS0zOGMwLTRkYjctYWZjMC1jMzIzZTRjYjBhZDMiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.J-fCCyQazXrlzhgwcgYzyZIRZpezXo4_8WvoupBSktmwh

<h2>Configure Agentcore Runtime Deployment</h2>

In [3]:
import os
os.environ["AWS_PROFILE"] = "agentcore"

from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['forecast.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="forecast.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="mcp_server_agentcore"
)
print("Configuration completed ✓")

Entrypoint parsed: file=C:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\forecast.py, bedrock_agentcore_name=forecast
Configuring BedrockAgentCore agent: mcp_server_agentcore


Using AWS region: us-east-1
All required files found ✓
Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\Dockerfile
Generated .dockerignore: c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\.dockerignore
Setting 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: c:\Users\ZALIZANZ\Documents\personal\housing-forecast-poc\forecaster\.bedrock_agentcore.yaml


Configuration completed ✓


<h2>Deploy MCP Server to AgentCore Runtime</h2>

In [4]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 966080101830 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 966080101830 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_agentcore


Launching MCP server to AgentCore Runtime...
This may take several minutes...


✅ ECR repository available: 966080101830.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_agentcore
Getting or creating execution role for agent: mcp_server_agentcore
Using AWS region: us-east-1, account ID: 966080101830
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4


✅ Reusing existing ECR repository: 966080101830.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_agentcore


✅ Reusing existing execution role: arn:aws:iam::966080101830:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
✅ Execution role available: arn:aws:iam::966080101830:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: mcp_server_agentcore
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-7f3ae149b4
Reusing existing CodeBuild execution role: arn:aws:iam::966080101830:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-7f3ae149b4
Using .dockerignore with 44 patterns
Uploaded source to S3: mcp_server_agentcore/20250903-081242.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_agentcore-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.3s
🔄 PROVISIONING started (total: 6s)
✅ PROVISIONING completed in 10.6s
🔄 PRE_BUILD started (total: 16s)
✅ PRE_BUILD com

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:966080101830:runtime/mcp_server_agentcore-S1bHZKAQcK
Agent ID: mcp_server_agentcore-S1bHZKAQcK


<h2>Checking Agentcore Runtime Status</h2>

In [5]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

Checking AgentCore Runtime status...


Retrieved Bedrock AgentCore status for: mcp_server_agentcore


Initial status: READY
✓ AgentCore Runtime is READY!
Final status: READY


<h2>Store Configuration in Parameter Store and Secret Manager For Remote Access</h2>

In [6]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials stored in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:966080101830:runtime/mcp_server_agentcore-S1bHZKAQcK


<h2>Clean Up</h2>

In [7]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

🗑️  Starting cleanup process...
Deleting AgentCore Runtime...
✓ AgentCore Runtime deletion initiated
Deleting ECR repository...
✓ ECR repository deleted
✓ Parameter Store parameter deleted
✓ Secrets Manager secret deleted

✅ Cleanup completed successfully!
